In [2]:
import pandas as pd
import ast
from data_process import MLPDataset_For_League, make_champ_idx
from torch.utils.data import DataLoader
from BERT import BERT,BERTLM
# from WinnerPredictModel import Winner_Predictor, Winner_Predictor_Trainer
from CNN import CNN, CNN_Trainer
import torch

In [31]:
make_champ_idx("../Data/match_data_2.csv", 6)

{1: 6,
 2: 7,
 3: 8,
 4: 9,
 5: 10,
 6: 11,
 7: 12,
 8: 13,
 9: 14,
 10: 15,
 11: 16,
 12: 17,
 13: 18,
 14: 19,
 15: 20,
 16: 21,
 17: 22,
 18: 23,
 19: 24,
 20: 25,
 21: 26,
 22: 27,
 23: 28,
 24: 29,
 25: 30,
 26: 31,
 27: 32,
 28: 33,
 29: 34,
 30: 35,
 31: 36,
 32: 37,
 33: 38,
 34: 39,
 35: 40,
 36: 41,
 37: 42,
 38: 43,
 39: 44,
 40: 45,
 41: 46,
 42: 47,
 43: 48,
 44: 49,
 45: 50,
 48: 51,
 50: 52,
 51: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 67: 66,
 68: 67,
 69: 68,
 72: 69,
 74: 70,
 75: 71,
 76: 72,
 77: 73,
 78: 74,
 79: 75,
 80: 76,
 81: 77,
 82: 78,
 83: 79,
 84: 80,
 85: 81,
 86: 82,
 89: 83,
 90: 84,
 91: 85,
 92: 86,
 96: 87,
 98: 88,
 99: 89,
 101: 90,
 102: 91,
 103: 92,
 104: 93,
 105: 94,
 106: 95,
 107: 96,
 110: 97,
 111: 98,
 112: 99,
 113: 100,
 114: 101,
 115: 102,
 117: 103,
 119: 104,
 120: 105,
 121: 106,
 122: 107,
 126: 108,
 127: 109,
 131: 110,
 133: 111,
 134: 112,
 136: 113,
 14

In [82]:
df = pd.read_csv("../Data/match_data_new.csv")


In [83]:
# df["user_ids"] = df["user_ids"].apply(lambda x : ast.literal_eval(x))

In [75]:
# df[df['user_ids'].apply(lambda x: len(x) < 10)].index

Index([21599, 21600, 21601, 21602, 21603, 29317, 29318, 29319, 29320, 29321], dtype='int64')

In [79]:
# df = df.drop(df[df['user_ids'].apply(lambda x: len(x) < 10)].index).reset_index()

In [80]:
# df.to_csv("../Data/match_data_new.csv", index = False)

In [84]:
tokenizing_ids = {}

In [85]:
for team_ids in df["user_ids"]:
    ids = ast.literal_eval(team_ids)
    for id in ids:
        if id not in tokenizing_ids and len(tokenizing_ids) < 30000:
            tokenizing_ids[id] = len(tokenizing_ids) + 173
    

In [86]:
tokenizing_df = pd.DataFrame({"id": tokenizing_ids.keys(), "token" : tokenizing_ids.values()})

In [87]:
tokenizing_df.reset_index()
tokenizing_df.set_index("id")

,token
id,
klPaul124#NA1,173
Regdor#NA1,174
Remi#duck,175
Chase Groggy#booty,176
Cam Com#NA1,177
...,...
Fries#99999,30168
THE DRlVER#NA1,30169
ImpairedHeart#NA1,30170


In [88]:
tokenizing_df.to_csv("../Tokenizing/user_id_tokens.csv", index = False)

In [89]:
temp = pd.read_csv("../Tokenizing/user_id_tokens.csv", index_col = 'id')

In [90]:
temp

,token
id,
klPaul124#NA1,173
Regdor#NA1,174
Remi#duck,175
Chase Groggy#booty,176
Cam Com#NA1,177
...,...
Fries#99999,30168
THE DRlVER#NA1,30169
ImpairedHeart#NA1,30170


In [28]:
df = pd.read_csv("../Data/match_data_validation.csv")
df["teams"] = df["teams"].apply(lambda x : ast.literal_eval(x))


champ_idx = {}
with open("champ_idx.txt", 'r') as file:
    for line in file:
        pair = ast.literal_eval(line)
        champ_idx.update({pair[0]: pair[1]})



for i in range(len(df["teams"])):
    for j in range(len(df["teams"][i])):
        df["teams"][i][j] = champ_idx[df["teams"][i][j]]


train_datas = [[df["teams"][i][:5], df["teams"][i][5:], df["winner"][i]] for i in range(len(df))]

In [29]:
MAX_LEN = 13
vocab_size = 171

train_data = MLPDataset_For_League(
train_datas, seq_len=MAX_LEN)

test_datas = {"bert_input": [], "segment_label": [], "winner_label": []}
for data in train_data:
    for idx, d in data.items():
        test_datas[idx].append(d)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_datas = {key: torch.stack(value, dim = 0).to(device) for key, value in test_datas.items()}


# train_loader = DataLoader(
# train_data, batch_size=16, shuffle=True, pin_memory=True)


In [37]:
bert_model = BERT(
vocab_size=vocab_size,
d_model= 32,
n_layers=8,
heads=8,
dropout=0.1,
device = device
)

bert_ = BERTLM(bert_model, vocab_size)
bert_.load_state_dict(torch.load("../Trained_Model/bert_model_5"))


cn = CNN(bert_).to(device)
cn.load_state_dict(torch.load("../Trained_Model/cnn_model4"))

cn.eval()

winner_output = cn.forward(test_datas["bert_input"], test_datas["segment_label"])

# correct = winner_output.argmax(dim=-1).eq(test_datas["winner_label"]).sum().item()
correct = torch.round(torch.flatten(winner_output)).eq(test_datas["winner_label"]).sum().item()
total = correct/len(winner_output)
total

0.511

In [59]:

wpm = Winner_Predictor(bert_).to(device)
wpm.load_state_dict(torch.load("../Trained_Model/mlp_model"))

wpm.eval()

winner_output = wpm.forward(test_datas["bert_input"], test_datas["segment_label"])

# correct = winner_output.argmax(dim=-1).eq(test_datas["winner_label"]).sum().item()
correct = torch.round(torch.flatten(winner_output)).eq(test_datas["winner_label"]).sum().item()
total = correct/len(winner_output)
total

0.5086666666666667

In [5]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

bert_.eval()
x = (bert_.embedding(test_datas["bert_input"], test_datas["segment_label"])).to("cpu")
embedded_x = torch.cat((x[:, 1:6], x[:, 7:-1]), dim = 1)
embedded_x = torch.flatten(embedded_x, start_dim=1)

from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(embedded_x, test_datas["winner_label"].to("cpu"))


# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [8]:
x = (bert_.embedding(test_datas["bert_input"], test_datas["segment_label"])).to("cpu")
embedded_x = torch.cat((x[:, 1:6], x[:, 7:-1]), dim = 1)
embedded_x = torch.flatten(embedded_x, start_dim=1)
clf.score(embedded_x, test_datas["winner_label"].to("cpu"))


0.5193333333333333

In [7]:
id = 111
print(winner_output[id], test_datas["winner_label"][id])

tensor([0.5836], device='cuda:0', grad_fn=<SelectBackward0>) tensor(0, device='cuda:0')
